### EXTRAÇÃO DAS BASES DE DADOS RAW DO ORACLE E MANIPULAÇÕES DE CLEASING PARA POSTERIOR ESCRITA EM S3

CENTRAL_RISCO_LIMITE
LYDIANS.INFCARTEIRAMENSAL

In [1]:
import boto3
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import regexp_replace
from pyspark import SparkConf
from pyspark.sql import SparkSession

import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains,round
from functools import reduce
from operator import add, and_, sub

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1644837942000_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
schema = StructType() \
      .add("DATA_BASE",StringType(),True) \
      .add("CPF_CLIENTE",StringType(),True) \
      .add("CONTRATO",StringType(),True) \
      .add("DETCLI",StringType(),True) \
      .add("MOD_",StringType(),True) \
      .add("COSIF",StringType(),True) \
      .add("ORIGEMREC",StringType(),True) \
      .add("INDX",StringType(),True) \
      .add("PERCINDX",StringType(),True) \
      .add("VARCAMB",StringType(),True) \
      .add("CEP",StringType(),True) \
      .add("TAXEFT",StringType(),True) \
      .add("DT_CONTRATO",StringType(),True) \
      .add("VLR_CONTR",StringType(),True) \
      .add("NATUREZA_OPERACAO",StringType(),True) \
      .add("DT_VENC",StringType(),True) \
      .add("CLASSE_OPERACAO",StringType(),True) \
      .add("PROVCONSTTD",StringType(),True) \
      .add("DIAATRASO",StringType(),True) \
      .add("CARACESPECIAL",StringType(),True) \
      .add("VCOD",StringType(),True) \
      .add("VALOR",StringType(),True) \
      .add("CNPJ_EMP",StringType(),True) \
      .add("PROCESSAMENTO",StringType(),True) \
      .add("INIRELACCLI",StringType(),True) \
      .add("PORTE_CLIENTE",StringType(),True) \
      .add("NIVEL_RISCO_MINIMO",StringType(),True) \
      .add("FATURAMENTO_ANUAL",StringType(),True) \
      .add("CONG_ECON",StringType(),True) \
      .add("TPCLI",StringType(),True) \
      .add("TPCTRL",StringType(),True) \
      .add("IPOC",StringType(),True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def sumIgnoringNaN(listOfColumnsToSum):
    return F.when(
        reduce(and_,
               [(F.col(column).isNull()) for column in listOfColumnsToSum]),
        F.lit(None)).otherwise(
            reduce(add, [
                F.coalesce(F.col(column), F.lit(0.0))
                for column in listOfColumnsToSum
            ]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Leitura da base CENTRAL_RISCO_LIMITE
df = spark.read.format("csv") \
      .options(header='True', delimiter=';') \
      .schema(schema) \
      .load('s3://modelagem/scr/extracoes-central-risco-limite/')

for column in df.columns:
    if column.startswith('LIMITE') or column.startswith('CRED'):
        df = df.withColumn(column, F.regexp_replace(column, ',','.')).withColumn(column, F.col(column).cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df1 = df.where(col("DATA_BASE").isNotNull()).orderBy("CPF_CLIENTE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading all dfs from S3 bucket:

In [6]:
df1 = df1.na.fill(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Appending to one dataframe:

In [7]:
df1 = df1.withColumn("VALOR",round(df1.VALOR.cast(DoubleType()),4))
df1 = df1.withColumn("FATURAMENTO_ANUAL",round(df1.FATURAMENTO_ANUAL.cast(DoubleType()),4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dict to rename columns:

In [8]:
renameCodVenc = {"LIMITE_CRED_VENCIMENTO_ATE_360":"v20","LIMITE_CRED_VENCIM_ACIMA_360":"v40","CRED_A_LIBERAR_ATE_360":"v60","CRED_A_LIBERAR_ACIMA_360":"v80","CRED_A_VENCER_ATE_30":"v110","CRED_A_VENCER_DE_31_A_60":"v120","CRED_A_VENCER_DE_61_A_90":"v130","CRED_A_VENCER_DE_91_A_180":"v140","CRED_A_VENCER_DE_181_A_360":"v150","CRED_A_VENCER_DE_361_A_720":"v160","CRED_A_VENCER_DE_721_A_1080":"v165","CRED_A_VENCER_DE_1081_A_1440":"v170","CRED_A_VENCER_DE_1441_A_1800":"v175","CRED_A_VENCER_DE_1801_A_5400":"v180","CRED_A_VENCER_ACIMA_5400":"v190","CRED_A_VENCER_PRAZO_INDETERM":"v199","CRED_VENCIDOS_DE_1_A_14":"v205","CRED_VENCIDOS_DE_15_A_30":"v210","CRED_VENCIDOS_DE_31_A_60":"v220","CRED_VENCIDOS_DE_61_A_90":"v230","CRED_VENCIDOS_DE_91_A_120":"v240","CRED_VENCIDOS_DE_121_A_150":"v245","CRED_VENCIDOS_DE_151_A_180":"v250","CRED_VENCIDOS_DE_181_A_240":"v255","CRED_VENCIDOS_DE_241_A_300":"v260","CRED_VENCIDOS_DE_301_A_360":"v270","CRED_VENCIDOS_DE_361_A_540":"v280","CRED_VENCIDOS_ACIMA_540":"v290","CRED_BAIX_PREJ_ATE_12_M":"v310","CRED_BAIX_PREJ_MAIS_12_A_48_M":"v320","CRED_BAIX_PREJ_MAIS_48_M":"v330"}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
codVencList = list(renameCodVenc.values())

df1 = df1.groupBy('DATA_BASE','CPF_CLIENTE','MOD_').pivot('VCOD', codVencList).sum('VALOR')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Renaming columns:

In [10]:
df1 = df1.withColumnRenamed('CPF_CLIENTE','CPF_CNPJ')
df1 = df1.withColumnRenamed('MOD_','MODALIDADE_OPERACAO')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
for key in renameCodVenc:
    df1 = df1.withColumnRenamed(renameCodVenc[key],key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saving it in S3:

In [12]:
df1.write.format("parquet").mode("append").save("s3://modelagem/scr/omni-scr/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
schema_ = StructType() \
      .add("SEQUENCIA",StringType(),True)\
      .add("EMPRESA",StringType(),True)\
      .add("CODSISTEMA",StringType(),True)\
      .add("DTMOVIMENTO",StringType(),True)\
      .add("AGENCIA",StringType(),True)\
      .add("NROOPERACAO",StringType(),True)\
      .add("CLIENTE",StringType(),True)\
      .add("CODPRODUTO",StringType(),True)\
      .add("CODGERENTE",StringType(),True)\
      .add("SLDCONTABIL",StringType(),True)\
      .add("SLDRENDA",StringType(),True)\
      .add("SLDCURVA",StringType(),True)\
      .add("SLDCOOBRIGACOES",StringType(),True)\
      .add("FAIXARISCO",StringType(),True)\
      .add("DTVCTO",StringType(),True)\
      .add("DTPDD",StringType(),True)\
      .add("DOCUMENTO",StringType(),True)\
      .add("TPPESSOA",StringType(),True)\
      .add("DOCCENTRALRISCO",StringType(),True)\
      .add("FAIXAATRASO",StringType(),True)\
      .add("FAIXAA",StringType(),True)\
      .add("FAIXAPDD",StringType(),True)\
      .add("FAIXAB",StringType(),True)\
      .add("FAIXAC",StringType(),True)\
      .add("FAIXAD",StringType(),True)\
      .add("FAIXAE",StringType(),True)\
      .add("FAIXAF",StringType(),True)\
      .add("FAIXAG",StringType(),True)\
      .add("PARCVENCIDA",StringType(),True)\
      .add("PARCAVENCER14",StringType(),True)\
      .add("PARCAVENCER91",StringType(),True)\
      .add("PARCAVENCER361",StringType(),True)\
      .add("PARCAVENCER1081",StringType(),True)\
      .add("PARCAVENCER1801",StringType(),True)\
      .add("PARCAVENCER5400",StringType(),True)\
      .add("DTEMISSAO",StringType(),True)\
      .add("DIRECIONAMENTO",StringType(),True)\
      .add("SETOR",StringType(),True)\
      .add("IDPDD",StringType(),True)\
      .add("PROVISAO",StringType(),True)\
      .add("RISCOPOLITICA",StringType(),True)\
      .add("RISCOPROCESSADO",StringType(),True)\
      .add("RISCOANTERIOR",StringType(),True)\
      .add("DTCADASTRAL",StringType(),True)\
      .add("CONGLOMERADO",StringType(),True)\
      .add("ORIGEMRECURSO",StringType(),True)\
      .add("LOCALIZACAO",StringType(),True)\
      .add("MODALIDADE",StringType(),True)\
      .add("NATOPERACAO",StringType(),True)\
      .add("CARACTESPECIAL",StringType(),True)\
      .add("CARACTESPECIAL_A",StringType(),True)\
      .add("TPRENEGOCIACAO",StringType(),True)\
      .add("DTRENEGOCIACAO",StringType(),True)\
      .add("TPGARANTIA",StringType(),True)\
      .add("VLRGARANTIA",StringType(),True)\
      .add("DTGARANTIAAVALIACAO",StringType(),True)\
      .add("VLRTOTDESCONTO",StringType(),True)\
      .add("VLRTOTDESCONTOLIQ",StringType(),True)\
      .add("AVENCER30",StringType(),True)\
      .add("AVENCER31_60",StringType(),True)\
      .add("AVENCER61_90",StringType(),True)\
      .add("AVENCER91_180",StringType(),True)\
      .add("AVENCER181_360",StringType(),True)\
      .add("AVENCER361_720",StringType(),True)\
      .add("AVENCER721_1080",StringType(),True)\
      .add("AVENCER1081_1440",StringType(),True)\
      .add("AVENCER1441_1800",StringType(),True)\
      .add("AVENCER1801_5400",StringType(),True)\
      .add("AVENCER5401",StringType(),True)\
      .add("AVENCERINDETERMINADO",StringType(),True)\
      .add("VENCIDO1_14",StringType(),True)\
      .add("VENCIDO15_30",StringType(),True)\
      .add("VENCIDO31_60",StringType(),True)\
      .add("VENCIDO61_90",StringType(),True)\
      .add("VENCIDO91_120",StringType(),True)\
      .add("VENCIDO121_150",StringType(),True)\
      .add("VENCIDO151_180",StringType(),True)\
      .add("VENCIDO181_240",StringType(),True)\
      .add("VENCIDO241_300",StringType(),True)\
      .add("VENCIDO301_360",StringType(),True)\
      .add("VENCIDO361_540",StringType(),True)\
      .add("VENCIDO541",StringType(),True)\
      .add("PREJUIZO12",StringType(),True)\
      .add("PREJUIZO12_48",StringType(),True)\
      .add("PREJUIZO49",StringType(),True)\
      .add("TXCONTRATUAL",StringType(),True)\
      .add("CEPAGENCIA",StringType(),True)\
      .add("INDEXADOR",StringType(),True)\
      .add("VARCAMBIAL",StringType(),True)\
      .add("CONTRATORENEG",StringType(),True)\
      .add("PRAZODOBRO",StringType(),True)\
      .add("GARANTIDOR",StringType(),True)\
      .add("DTVENCFINAL",StringType(),True)\
      .add("PORTECLIENTE",StringType(),True)\
      .add("AUTORIZACONSBC",StringType(),True)\
      .add("MODRENEGOCIACAO",StringType(),True)\
      .add("SEQ",StringType(),True)\
      .add("RISCOSISTEMA",StringType(),True)\
      .add("IDCESSAONOATIVO",StringType(),True)\
      .add("TPCONTROLE",StringType(),True)\
      .add("IDCESSAOPARCIAL",StringType(),True) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Leitura da base infcarteiramensal
df_ = spark.read.format("csv") \
      .options(header='True', delimiter=';') \
      .schema(schema_) \
      .load('s3://modelagem/scr/extracoes-infcarteiramensal/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_ = df_.withColumn("LIMITE_CRED_VENCIMENTO_ATE_360",lit(0.0))
df_ = df_.withColumn("LIMITE_CRED_VENCIM_ACIMA_360",lit(0.0))
df_ = df_.withColumn("CRED_A_LIBERAR_ATE_360",lit(0.0))
df_ = df_.withColumn("CRED_A_LIBERAR_ACIMA_360",lit(0.0))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
renameCodVenc_ = {"CRED_A_VENCER_ATE_30":"AVENCER30","CRED_A_VENCER_DE_31_A_60":"AVENCER31_60","CRED_A_VENCER_DE_61_A_90":"AVENCER61_90","CRED_A_VENCER_DE_91_A_180":"AVENCER91_180","CRED_A_VENCER_DE_181_A_360":"AVENCER181_360", \
"CRED_A_VENCER_DE_361_A_720":"AVENCER361_720","CRED_A_VENCER_DE_721_A_1080":"AVENCER721_1080","CRED_A_VENCER_DE_1081_A_1440":"AVENCER1081_1440","CRED_A_VENCER_DE_1441_A_1800":"AVENCER1441_1800","CRED_A_VENCER_DE_1801_A_5400":"AVENCER1801_5400",\
"CRED_A_VENCER_ACIMA_5400":"AVENCER5401","CRED_A_VENCER_PRAZO_INDETERM":"AVENCERINDETERMINADO","CRED_VENCIDOS_DE_1_A_14":"VENCIDO1_14","CRED_VENCIDOS_DE_15_A_30":"VENCIDO15_30","CRED_VENCIDOS_DE_31_A_60":"VENCIDO31_60","CRED_VENCIDOS_DE_61_A_90":"VENCIDO61_90",\
"CRED_VENCIDOS_DE_91_A_120":"VENCIDO91_120","CRED_VENCIDOS_DE_121_A_150":"VENCIDO121_150","CRED_VENCIDOS_DE_151_A_180":"VENCIDO151_180","CRED_VENCIDOS_DE_181_A_240":"VENCIDO181_240","CRED_VENCIDOS_DE_241_A_300":"VENCIDO241_300",\
"CRED_VENCIDOS_DE_301_A_360":"VENCIDO301_360","CRED_VENCIDOS_DE_361_A_540":"VENCIDO361_540","CRED_VENCIDOS_ACIMA_540":"VENCIDO541","CRED_BAIX_PREJ_ATE_12_M":"PREJUIZO12","CRED_BAIX_PREJ_MAIS_12_A_48_M":"PREJUIZO12_48","CRED_BAIX_PREJ_MAIS_48_M":"PREJUIZO49"}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
for key in renameCodVenc_:
    df_ = df_.withColumnRenamed(renameCodVenc_[key],key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_ = df_.withColumnRenamed('DOCUMENTO','CPF_CNPJ')
df_ = df_.withColumnRenamed('MODALIDADE','MODALIDADE_OPERACAO')
df_ = df_.withColumnRenamed('DTMOVIMENTO','DATA_BASE')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
lista_soma = ["LIMITE_CRED_VENCIMENTO_ATE_360",
"LIMITE_CRED_VENCIM_ACIMA_360",
"CRED_A_LIBERAR_ATE_360",
"CRED_A_LIBERAR_ACIMA_360",
"CRED_A_VENCER_ATE_30",
"CRED_A_VENCER_DE_31_A_60",
"CRED_A_VENCER_DE_61_A_90",
"CRED_A_VENCER_DE_91_A_180",
"CRED_A_VENCER_DE_181_A_360",
"CRED_A_VENCER_DE_361_A_720",
"CRED_A_VENCER_DE_721_A_1080",
"CRED_A_VENCER_DE_1081_A_1440",
"CRED_A_VENCER_DE_1441_A_1800",
"CRED_A_VENCER_DE_1801_A_5400",
"CRED_A_VENCER_ACIMA_5400",
"CRED_A_VENCER_PRAZO_INDETERM",
"CRED_VENCIDOS_DE_1_A_14",
"CRED_VENCIDOS_DE_15_A_30",
"CRED_VENCIDOS_DE_31_A_60",
"CRED_VENCIDOS_DE_61_A_90",
"CRED_VENCIDOS_DE_91_A_120",
"CRED_VENCIDOS_DE_121_A_150",
"CRED_VENCIDOS_DE_151_A_180",
"CRED_VENCIDOS_DE_181_A_240",
"CRED_VENCIDOS_DE_241_A_300",
"CRED_VENCIDOS_DE_301_A_360",
"CRED_VENCIDOS_DE_361_A_540",
"CRED_VENCIDOS_ACIMA_540",
"CRED_BAIX_PREJ_ATE_12_M",
"CRED_BAIX_PREJ_MAIS_12_A_48_M",
"CRED_BAIX_PREJ_MAIS_48_M"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
for column in df_.columns:
    if column.startswith('LIMITE') or column.startswith('CRED'):
        df_ = df_.withColumn(column, F.regexp_replace(column, ',','.')).withColumn(column, F.col(column).cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_ = df_.groupBy('DATA_BASE','CPF_CNPJ','MODALIDADE_OPERACAO').sum("LIMITE_CRED_VENCIMENTO_ATE_360",
"LIMITE_CRED_VENCIM_ACIMA_360",
"CRED_A_LIBERAR_ATE_360",
"CRED_A_LIBERAR_ACIMA_360",
"CRED_A_VENCER_ATE_30",
"CRED_A_VENCER_DE_31_A_60",
"CRED_A_VENCER_DE_61_A_90",
"CRED_A_VENCER_DE_91_A_180",
"CRED_A_VENCER_DE_181_A_360",
"CRED_A_VENCER_DE_361_A_720",
"CRED_A_VENCER_DE_721_A_1080",
"CRED_A_VENCER_DE_1081_A_1440",
"CRED_A_VENCER_DE_1441_A_1800",
"CRED_A_VENCER_DE_1801_A_5400",
"CRED_A_VENCER_ACIMA_5400",
"CRED_A_VENCER_PRAZO_INDETERM",
"CRED_VENCIDOS_DE_1_A_14",
"CRED_VENCIDOS_DE_15_A_30",
"CRED_VENCIDOS_DE_31_A_60",
"CRED_VENCIDOS_DE_61_A_90",
"CRED_VENCIDOS_DE_91_A_120",
"CRED_VENCIDOS_DE_121_A_150",
"CRED_VENCIDOS_DE_151_A_180",
"CRED_VENCIDOS_DE_181_A_240",
"CRED_VENCIDOS_DE_241_A_300",
"CRED_VENCIDOS_DE_301_A_360",
"CRED_VENCIDOS_DE_361_A_540",
"CRED_VENCIDOS_ACIMA_540",
"CRED_BAIX_PREJ_ATE_12_M",
"CRED_BAIX_PREJ_MAIS_12_A_48_M",
"CRED_BAIX_PREJ_MAIS_48_M") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
for column in df_.columns:
    df_ = df_.withColumnRenamed(column, column.replace('sum(','').replace(')',''))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
for item in lista_soma:
    df_ = df_.withColumn(item, F.round(df_[item], 4))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_ = df_.select(
"DATA_BASE",
"CPF_CNPJ",
"MODALIDADE_OPERACAO",
"LIMITE_CRED_VENCIMENTO_ATE_360",
"LIMITE_CRED_VENCIM_ACIMA_360",
"CRED_A_LIBERAR_ATE_360",
"CRED_A_LIBERAR_ACIMA_360",
"CRED_A_VENCER_ATE_30",
"CRED_A_VENCER_DE_31_A_60",
"CRED_A_VENCER_DE_61_A_90",
"CRED_A_VENCER_DE_91_A_180",
"CRED_A_VENCER_DE_181_A_360",
"CRED_A_VENCER_DE_361_A_720",
"CRED_A_VENCER_DE_721_A_1080",
"CRED_A_VENCER_DE_1081_A_1440",
"CRED_A_VENCER_DE_1441_A_1800",
"CRED_A_VENCER_DE_1801_A_5400",
"CRED_A_VENCER_ACIMA_5400",
"CRED_A_VENCER_PRAZO_INDETERM",
"CRED_VENCIDOS_DE_1_A_14",
"CRED_VENCIDOS_DE_15_A_30",
"CRED_VENCIDOS_DE_31_A_60",
"CRED_VENCIDOS_DE_61_A_90",
"CRED_VENCIDOS_DE_91_A_120",
"CRED_VENCIDOS_DE_121_A_150",
"CRED_VENCIDOS_DE_151_A_180",
"CRED_VENCIDOS_DE_181_A_240",
"CRED_VENCIDOS_DE_241_A_300",
"CRED_VENCIDOS_DE_301_A_360",
"CRED_VENCIDOS_DE_361_A_540",
"CRED_VENCIDOS_ACIMA_540",
"CRED_BAIX_PREJ_ATE_12_M",
"CRED_BAIX_PREJ_MAIS_12_A_48_M",
"CRED_BAIX_PREJ_MAIS_48_M"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_.write.format("parquet").mode("append").save("s3://modelagem/scr/omni-scr/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…